<a href="https://colab.research.google.com/github/TheUnmeshRaj/Advanced-Generative-AI-For-Skin-Diseases/blob/main/Final_Model_for_Skin_Disease_Treatment_Plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers torch scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer1 = AutoTokenizer.from_pretrained("Unmeshraj/skin-disease-detection")
model1 = AutoModel.from_pretrained("Unmeshraj/skin-disease-detection")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer2 = AutoTokenizer.from_pretrained("Unmeshraj/skin-disease-treatment-plan")
model2 = AutoModel.from_pretrained("Unmeshraj/skin-disease-treatment-plan")

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, GenerationConfig
import torch
from sklearn.metrics.pairwise import cosine_similarity

queries = []
diseases = []
embeddings = []

dataset = load_dataset("Mostafijur/Skin_disease_classify_data")

def embed_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

for example in dataset['train']:
    query = example['Skin_disease_classification']['query']
    disease = example['Skin_disease_classification']['disease']
    queries.append(query)
    diseases.append(disease)

    query_embedding = embed_text(query, tokenizer1, model1)
    embeddings.append(query_embedding)



def find_similar_disease(input_query, queries, embeddings, tokenizer, model):
    input_embedding = embed_text(input_query, tokenizer, model)
    similarities = [cosine_similarity(input_embedding.detach().numpy(), emb.detach().numpy())[0][0] for emb in embeddings]
    most_similar_idx = similarities.index(max(similarities))
    return diseases[most_similar_idx]


In [7]:
topics = []
information = []
topic_embeddings = []

dataset1 = load_dataset("brucewayne0459/Skin_diseases_and_care")

for example in dataset1['train']:
    topic = example['Topic']
    info = example['Information']
    topics.append(topic)
    information.append(info)

    topic_embedding = embed_text(topic, tokenizer2, model2)
    topic_embeddings.append(topic_embedding)

def find_treatment_plan(disease_name, topics, topic_embeddings, tokenizer2, model2):
    disease_embedding = embed_text(disease_name, tokenizer2, model2)
    similarities = [cosine_similarity(disease_embedding.detach().numpy(), topic_emb.detach().numpy())[0][0] for topic_emb in topic_embeddings]
    most_similar_idx = similarities.index(max(similarities))
    return information[most_similar_idx]

README.md:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/477k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/828 [00:00<?, ? examples/s]

In [10]:
input_query = input("Enter your Symptons here: ")
similar_disease = find_similar_disease(input_query, queries, embeddings, tokenizer1, model1)
treatment_plan = find_treatment_plan(similar_disease, topics, topic_embeddings, tokenizer2, model2)
print(f"Treatment Plan for your similar disease: {treatment_plan}")

Enter your Symptons here: I have these small, red spots on my face that get worse with sun exposure. They’re not itchy but look unsightly. 
Treatment Plan for your similar disease: Lupus is an autoimmune disease where the immune system attacks healthy tissues.  Several types of lupus exist, each affecting different parts of the body, including the skin. Systemic lupus erythematosus (SLE) is the most common type and is more frequent in women, particularly Black American women. Discoid lupus erythematosus (DLE), affecting only the skin, is most prevalent in Black people.

While the exact cause is unknown, research suggests that lupus develops due to a combination of genetic predisposition and environmental triggers. Genetic variants affecting the immune system play a crucial role, and several genes associated with lupus have been identified. 

Environmental triggers can include ultraviolet (UV) light exposure from sunlight, tanning beds, and fluorescent bulbs, cigarette smoke, infections

In [11]:
!pip install transformers huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

NameError: name 'model' is not defined

In [12]:
model1.save_pretrained('skin-disease-treatment-promax')
tokenizer1.save_pretrained('skin-disease-treatment-promax')

from huggingface_hub import HfApi

repo_name = "Unmeshraj/skin-disease-treatment-promax"

model1.push_to_hub(repo_name)
tokenizer1.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Unmeshraj/skin-disease-treatment-promax/commit/47d0f50ead4b6ddfb8c5410c418e82a8f51fb266', commit_message='Upload tokenizer', commit_description='', oid='47d0f50ead4b6ddfb8c5410c418e82a8f51fb266', pr_url=None, pr_revision=None, pr_num=None)